In [ ]:
# Install MathWriting dataset 

In [48]:
%%capture
%run ./config.ipynb
%run InkML-parser.ipynb

PROJECT_DIR: /Users/ball/Documents/workspace/latex-generator
PROJECT_DIR: /Users/ball/Documents/workspace/latex-generator
MODEL_DIR: /Users/ball/Documents/workspace/latex-generator/models
MODEL_DIR: /Users/ball/Documents/workspace/latex-generator/models
LOG_DIR: /Users/ball/Documents/workspace/latex-generator/logs
LOG_DIR: /Users/ball/Documents/workspace/latex-generator/logs
TIME_SAMPLING_DELTA: 30
TIME_SAMPLING_DELTA: 30
EPOCHS: 100
EPOCHS: 100
BATCH_SIZE: 64
BATCH_SIZE: 64
IMG_SIZe: 256
IMG_SIZe: 256
PROJECT_DIR: /Users/ball/Documents/workspace/latex-generator
PROJECT_DIR: /Users/ball/Documents/workspace/latex-generator
MODEL_DIR: /Users/ball/Documents/workspace/latex-generator/models
MODEL_DIR: /Users/ball/Documents/workspace/latex-generator/models
LOG_DIR: /Users/ball/Documents/workspace/latex-generator/logs
LOG_DIR: /Users/ball/Documents/workspace/latex-generator/logs
TIME_SAMPLING_DELTA: 30
TIME_SAMPLING_DELTA: 30
EPOCHS: 100
EPOCHS: 100
BATCH_SIZE: 64
BATCH_SIZE: 64
IMG_SIZe: 25

In [46]:
import torch 
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import numpy as np
import time
import matplotlib.pyplot as plt

class MathWritingDataset(Dataset):
    def __init__(self, dataset_dir, data_types=["train", "symbols", "synthetic"], transform=None):
        self.dataset_dir = dataset_dir
        self.types = data_types
        self.filenames = []
        self.transform = transform
        for type_ in self.types:
            filename = [f'{type_}/{f.name}' for f in (self.dataset_dir / type_).glob("*.inkml")]
            self.filenames.extend(filename)
        
    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        # start = time.time()
        assert type(idx) == int
        target_file_path = self.dataset_dir / self.filenames[idx]

        # read inkml file 
        ink = read_inkml_file(target_file_path)

        # generate ink sequence 
        text_sequence = get_ink_sequence_token(ink, TIME_SAMPLING_DELTA)

        image = get_ink_image(ink, IMG_SIZE)
            

        label = ink.annotations['normalizedLabel']

        sample = {'image': image, 'text': text_sequence, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        # print(f'time taken to load one data: {time.time() - start}s')

        return sample


In [47]:
# TEST PURPOSE
tmp_dataset = MathWritingDataset(data_dir / "mathwriting-2024-excerpt")
print(tmp_dataset[0])

# show image 
# plt.axis('off')
# plt.imshow(tmp_dataset[0]['image'])

{'image': array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 

<Figure size 640x480 with 0 Axes>

In [49]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = MathWritingDataset(data_dir / "mathwriting-2024-excerpt", data_types=["train", "symbols", "synthetic"], transform=train_transform)
test_dataset = MathWritingDataset(data_dir / "mathwriting-2024-excerpt", data_types=["test"], transform=test_transform)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
